In [23]:
import os
import warnings
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz 
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

load_dotenv()

True

In [24]:
# Set Alpaca API key and secret
# YOUR CODE HERE!
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
# Create the Alpaca API object
# YOUR CODE HERE!
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [25]:
# Define the timezone: America/New_York
tz = pytz.timezone('America/New_York')

# Get the current datetime in the specified timezone
now = datetime.now(tz)
correct_date = now - timedelta(days=1)  # Use previous day if after market close

# Convert to pandas Timestamp with timezone and format as ISO 8601 string
correct_timestamp = pd.Timestamp(correct_date)
correct_timestamp_normalized = correct_timestamp.normalize()
correct_date_str = correct_timestamp_normalized.isoformat()

# Set the tickers and timeframe
tickers = ["AGG", "SPY", "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JNJ", "JPM"]
timeframe = "1Day"

# Fetch the data
current_portfolio_df = api.get_bars(tickers, timeframe, start=correct_date_str, end=correct_date_str).df
print("Data fetched successfully. Number of rows: ", len(current_portfolio_df))
print(current_portfolio_df.head())

Data fetched successfully. Number of rows:  0
Empty DataFrame
Columns: []
Index: []


In [26]:
if 'symbol' in current_portfolio_df.columns:
    # Separate data by ticker and drop the 'symbol' column
    agg_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AGG'].drop(columns='symbol', errors='ignore')
    spy_df = current_portfolio_df[current_portfolio_df['symbol'] == 'SPY'].drop(columns='symbol', errors='ignore')
    aapl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AAPL'].drop(columns='symbol', errors='ignore')
    msft_df = current_portfolio_df[current_portfolio_df['symbol'] == 'MSFT'].drop(columns='symbol', errors='ignore')
    amzn_df = current_portfolio_df[current_portfolio_df['symbol'] == 'AMZN'].drop(columns='symbol', errors='ignore')
    googl_df = current_portfolio_df[current_portfolio_df['symbol'] == 'GOOGL'].drop(columns='symbol', errors='ignore')
    meta_df = current_portfolio_df[current_portfolio_df['symbol'] == 'META'].drop(columns='symbol', errors='ignore')
    tsla_df = current_portfolio_df[current_portfolio_df['symbol'] == 'TSLA'].drop(columns='symbol', errors='ignore')
    nvda_df = current_portfolio_df[current_portfolio_df['symbol'] == 'NVDA'].drop(columns='symbol', errors='ignore')
    jnj_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JNJ'].drop(columns='symbol', errors='ignore')
    jpm_df = current_portfolio_df[current_portfolio_df['symbol'] == 'JPM'].drop(columns='symbol', errors='ignore')

    # Concatenate the ticker DataFrames along the columns axis
    portfolio_concat_df = pd.concat([agg_df, spy_df, aapl_df, msft_df, amzn_df, googl_df, meta_df, tsla_df, nvda_df, jnj_df, jpm_df],
                                    axis=1, 
                                    keys=['AGG', 'SPY', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JNJ', 'JPM'])
    print(portfolio_concat_df.head())
else:
    print("Symbol column not found in DataFrame.")

Symbol column not found in DataFrame.


In [27]:
import os
import pandas as pd
from datetime import datetime, timedelta
import pytz
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

# Load environment variables
load_dotenv()

# Print out the keys to verify they are loaded (remove in production)
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_API_SECRET_KEY")
print("API Key Loaded:", api_key)
print("Secret Key Loaded:", secret_key)

# Setup API credentials and create API object
if api_key and secret_key:
    api = tradeapi.REST(api_key, secret_key, api_version='v2')
    print("API client initialized successfully.")
else:
    print("API credentials are not loaded. Check .env file and ensure the notebook's working directory is correct.")

# Define the timezone and get the current date adjusted for NY market close
tz = pytz.timezone('America/New_York')
now = datetime.now(tz)
correct_date = now - timedelta(days=1)  # Use the previous day if after market close
correct_timestamp = pd.Timestamp(correct_date)
correct_timestamp_normalized = correct_timestamp.normalize()
correct_date_str = correct_timestamp_normalized.isoformat()

# Define the cryptocurrency tickers and the timeframe for the data request
crypto_tickers = ['BTCUSD', 'ETHUSD', 'BNBUSD', 'ADAUSD', 'XRPUSD', 'SOLUSD', 'DOTUSD', 'LTCUSD', 'LINKUSD', 'BCHUSD']
timeframe = '1Day'

# Fetch the cryptocurrency data using a method similar to get_bars() if supported
try:
    crypto_df = api.get_crypto_bars(crypto_tickers, timeframe, start=correct_date_str, end=correct_date_str).df
    print("Data fetched successfully. Number of rows:", len(crypto_df))
    print(crypto_df.head())

    if 'symbol' in crypto_df.columns:
        crypto_concat_df = pd.concat([crypto_df[crypto_df['symbol'] == ticker].drop(columns='symbol', errors='ignore') for ticker in crypto_tickers],
                                     axis=1, 
                                     keys=crypto_tickers)
        print(crypto_concat_df.head())
    else:
        print("Symbol column not found in cryptocurrency DataFrame. Check API response and documentation.")
except Exception as e:
    print("An error occurred while fetching or processing data:", str(e))


API Key Loaded: PKHRIE1T8QEDTBN7ONLV
Secret Key Loaded: None
API credentials are not loaded. Check .env file and ensure the notebook's working directory is correct.
An error occurred while fetching or processing data: invalid symbol: BTCUSD does not match ^[A-Z]+/[A-Z]+$


In [28]:
import os
import pandas as pd

# Define the directory path for raw_data
raw_data_dir = '../data/raw_data'
if not os.path.exists(raw_data_dir):
    os.makedirs(raw_data_dir)

# Function to save DataFrame to CSV or log if empty
def save_data(df, filename, description):
    if not df.empty:
        df.to_csv(filename)
        print(f"{description} data saved successfully to {filename}")
    else:
        print(f"No {description} data to save. Writing null placeholder.")
        # Write a null placeholder to indicate no data was fetched
        with open(filename, 'w') as file:
            file.write('null')

try:
    # Check if the DataFrame exists and isn't empty
    if 'portfolio_concat_df' in locals() and not portfolio_concat_df.empty:
        stock_file_path = os.path.join(raw_data_dir, 'stock_data.csv')
        save_data(portfolio_concat_df, stock_file_path, 'Stock')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'stock_data.csv'), 'Stock')

    if 'crypto_concat_df' in locals() and not crypto_concat_df.empty:
        crypto_file_path = os.path.join(raw_data_dir, 'crypto_data.csv')
        save_data(crypto_concat_df, crypto_file_path, 'Crypto')
    else:
        save_data(pd.DataFrame(), os.path.join(raw_data_dir, 'crypto_data.csv'), 'Crypto')
except Exception as e:
    print(f"An error occurred while saving data: {str(e)}")


Stock data saved successfully to ../data/raw_data/stock_data.csv
No Crypto data to save. Writing null placeholder.
